In [3]:
from datetime import datetime
import re
import pandas as pd
from datetime import datetime, timedelta
from unidecode import unidecode
df={'Origin':['BOGOTA','VILLAVICENCIO','BUCARAMANGA'],
    'Destination':['BOGOTA','VILLAVICENCIO','BUCARAMANGA'],
    'Cupo':['AAV837','DRZ986','FVN345'],
    'ESTADO':['MANTENIMIENTO','EN SERVICIO','MANTENIMIENTO'],
    'Fecha Incio':['09/13/2023','09/12/2023','09/10/2023'],
    'Fecha Finalizacion':['09/15/2023','09/12/2023','']   
    }

Mantenimiento=pd.read_excel('Mantenimiento.xlsx',header=0)
df_formato_guia = pd.read_excel('ACT_Formato_guia_control_renta.xlsx',header=0)
df_entrapetrol=pd.read_excel
columnas=['Origin','Destination','Cupo', 'ESTADO','Fecha Inicio','Fecha Finalizacion']
#print(f"{Mantenimiento}\n")
print(df_formato_guia.loc[df_formato_guia["ESTADO"]=="MANTENIMIENTO",columnas])
df=pd.DataFrame(df)

                    Origin        Destination    Cupo         ESTADO  \
2            VILLAVICENCIO      VILLAVICENCIO  FVQ475  MANTENIMIENTO   
7                 TRINIDAD           TRINIDAD  FVR345  MANTENIMIENTO   
8                 TRINIDAD              YOPAL  FVR348  MANTENIMIENTO   
10                TRINIDAD  TRINIDAD POZO IZA  FVS297  MANTENIMIENTO   
14                    SOPO             BOGOTA  LHT229  MANTENIMIENTO   
16               CARTAGENA          CARTAGENA  LHT624  MANTENIMIENTO   
17               CARTAGENA          CARTAGENA  LHT626  MANTENIMIENTO   
18                  BOGOTA               COTA  LHV855  MANTENIMIENTO   
25  DUITAMA/ HOTEL OLIMPIA             BOGOTA  FVR346  MANTENIMIENTO   

           Fecha Inicio Fecha Finalizacion  
2         4/3/2024 6:05                NaN  
7        3/26/2024 6:05                NaN  
8         4/5/2024 6:05                NaN  
10  2024-03-21 00:00:00                NaN  
14       4/16/2024 6:00     4/16/2024 7:30  
16       

In [4]:
# Fecha actual
hoy = datetime.now().date()
# Itera sobre las filas de df_formato_guia
for index, row in df_formato_guia.iterrows():
    if row['ESTADO'] == "MANTENIMIENTO":
        # Verifica si el Cupo existe en Mantenimiento
        if row['Cupo'] not in Mantenimiento['Cupo'].values:
            # Crea una nueva fila para agregar a Mantenimiento
            fecha_inicio = pd.to_datetime(row['Fecha Inicio']).date()  # Convert to datetime and extract date
            dias = (hoy - fecha_inicio).days
            dias = dias + 1
            nueva_fila = pd.DataFrame({'Cupo': [row['Cupo']], 'Inicio': [row['Fecha Inicio']],
                                       'Finalizo': [None], 'Dias': [dias], 'Observacion': [row['Observaciones']]})
            # Concatenar el nuevo DataFrame con Mantenimiento
            Mantenimiento = pd.concat([Mantenimiento, nueva_fila], ignore_index=True)
        else:
            # Busca la fila existente en Mantenimiento
            index_mantenimiento = Mantenimiento[Mantenimiento['Cupo'] == row['Cupo']].index[0]
            fecha_inicio_actual = pd.to_datetime(row['Fecha Inicio']).date()  # Convert to datetime and extract date
            fecha_inicio_mantenimiento = pd.to_datetime(Mantenimiento.loc[index_mantenimiento, 'Inicio']).date()  # Convert to datetime and extract date
            dias = (hoy - fecha_inicio_mantenimiento).days
            Mantenimiento.at[index_mantenimiento, 'Dias'] = dias + 1
            Mantenimiento.at[index_mantenimiento, 'Inicio'] = row['Fecha Inicio']  # Actualiza la columna 'Inicio'

    elif row['ESTADO'] != "MANTENIMIENTO":
        # Verifica si el Cupo existe en Mantenimiento y si no tiene fecha de finalización
        if row['Cupo'] in Mantenimiento['Cupo'].values and pd.isna(Mantenimiento[Mantenimiento['Cupo'] == row['Cupo']]['Finalizo'].values[0]):
            # Actualiza la fecha de finalización en Mantenimiento
            index_mantenimiento = Mantenimiento[Mantenimiento['Cupo'] == row['Cupo']].index[0]
            fecha_inicio_actual = pd.to_datetime(Mantenimiento.loc[Mantenimiento['Cupo'] == row['Cupo'], 'Inicio'].iloc[0]).date()  # Convert to datetime and extract date
            if row['ESTADO'] == 'DISPONIBLE':
                Mantenimiento.at[index_mantenimiento, 'Finalizo'] = hoy
            Mantenimiento.loc[index_mantenimiento, 'Finalizo'] = fecha_inicio_actual.strftime("%m/%d/%Y %H:%M:%S")
            dias = (hoy - fecha_inicio_actual).days
            Mantenimiento.at[index_mantenimiento, 'Dias'] = dias + 1

column_order = ['Cupo', 'Inicio', 'Finalizo', 'Dias', 'Observacion']
Mantenimiento = Mantenimiento[column_order]
Mantenimiento.to_excel('Mantenimiento.xlsx', index=False)
# Imprime el DataFrame Mantenimiento actualizado
print(Mantenimiento)


     Cupo               Inicio             Finalizo  Dias  \
0  FVS297  2024-03-21 00:00:00                  NaN    26   
1  FVQ475        4/3/2024 6:05                  NaN    13   
2  FVR345       3/26/2024 6:05                  NaN    21   
3  FVR348        4/5/2024 6:05                  NaN    11   
4  LHT229       4/16/2024 6:00                  NaN     0   
5  LHT624       4/15/2024 6:05                  NaN     1   
6  LHT626       4/15/2024 6:05                  NaN     1   
7  LHV855       4/15/2024 6:05                  NaN     1   
8  FVR346       2/21/2024 6:05  02/21/2024 00:00:00    55   

                          Observacion  
0  Movimiento en vacío retorno a base  
1                             GS15162  
2                                 NaN  
3                                 NaN  
4                             GS15161  
5                             GS15040  
6                             GS15119  
7                                 NaN  
8                            